# 第1章 引言

## 1.1 什么是语言模型 

从统计学习的角度切入，我们期望的是基于给定的文本信息输入，给出对应的新的文本/符号输出（可以是文本翻译、文本分类、文本扩写）。

大语言模型（Large Language Model，LLM）的核心原理基于深度学习和自然语言处理（NLP），尤其是Transformer架构。

大语言模型的关键目标是学习语言的概率分布，能够生成连贯的文本，理解复杂的语义和上下文。

要实现这样一个任务要解决两个问题：

1）输入序列：由于这里输入的是文本信号，而计算机能进入神经网络处理和计算的是数值，所以我们需要讲字符通过一定方式转化为数值。

2）输出序列：由于所需要输出的部分也是文本，而神经网络的输出是数值类型的（分类问题：二分类问题对应01输出，多分类对应多个01输出；回归问题：对应数值类型输出），所以需要建立神经网络的数值类型输出和最终字符输出的映射关系。



针对于第一个问题，其处理方式其实有很多种，比如最简单的，我们可以将输入序列进行编码，从而把字符转化为数值。

假设整个符号体系只有'a','b'两个小写字母和'.'一个句号。

输入序列是：'ab.b'

这里采用最简易的独热编码（可以搜索对应概念），总共3个字符，再加入'begin'和'end'两个字符串的起始和终止字符，其对应关系为

'begin':[0,0,0,0]; 'a':[1,0,0,0]; 'b':[0,1,0,0]; '.':[0,0,1,0]; 'end':[0,0,0,1]

则输入序列可以编码为:

\[[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,1,0,0],[0,0,0,1]]

然后此序列编码可以进入神经网络进行计算啦。

针对于第二个问题，同第一个问题类似，其处理方式其实有很多种，我们可以将输入序列进行编码，从而把数值转化为字符。

假设整个符号体系只有'a','b'两个小写字母和'.'一个句号。

输入序列是：'ab.b'，期望输出序列是'b.'

这里同样采用最简易的独热编码（可以搜索对应概念），总共4个字符，其对应关系为

[0,0,0,0]:'begin'; [1,0,0,0]:'a'; [0,1,0,0]:'b';  [0,0,1,0]:'.';  [0,0,0,1]:'end'; 

可以构建神经网络的输出是四分类，假设模型输出为\[0,0,1,0]，通过映射关系得到其输出字符'b'，将预测得到的结果传入模型，得到下一个输出 \[0,0,0,1]，通过映射关系得到其输出字符'.', 再将预测得到的结果传入模型，得到下一个输出 \[0,0,0,1]，通过映射关系得到其输出字符'end'，从而结束整个输出。 



语言模型（LM）的经典定义是一种对词元序列(token)的概率分布。假设我们有一个词元集的词汇表 $V$ 。语言模型p为每个词元序列 $x_{1},...,x_{L}$ ∈ $V$ 分配一个概率（介于0和1之间的数字）：

$$
p(x_1, \dots, x_L)
$$

概率直观地告诉我们一个标记序列有多“好（good）”。例如，如果词汇表为{ate, ball, cheese, mouse, the}，语言模型可能会分配以下概率（演示）：

$$
p(\text{the, mouse, ate, the, cheese}) = 0.02,
$$

$$
p(\text{the, cheese ate, the, mouse}) = 0.01,
$$

$$
p(\text{mouse, the, the, cheese, ate}) = 0.0001,
$$

从数学上讲，语言模型是一个非常简单而又美妙的对象。但是这种简单是具有欺骗性的：赋予所有序列以（有意义的）概率的能力，该能力要求语言模型具有非凡的（但是隐含的）语言能力和世界知识。

例如，语言模型应该隐含地赋予"𝗆𝗈𝗎𝗌𝖾 𝗍𝗁𝖾 𝗍𝗁𝖾 𝖼𝗁𝖾𝖾𝗌𝖾 𝖺𝗍𝖾"一个非常低的概率，因为它在语法上是不正确的（句法知识）。由于世界知识的存在，语言模型应该隐含地赋予"𝗍𝗁𝖾 𝗆𝗈𝗎𝗌𝖾 𝖺𝗍𝖾 𝗍𝗁𝖾 𝖼𝗁𝖾𝖾𝗌𝖾"比"𝗍𝗁𝖾 𝖼𝗁𝖾𝖾𝗌𝖾 𝖺𝗍𝖾 𝗍𝗁𝖾 𝗆𝗈𝗎𝗌𝖾"更高的概率。这是因为两个句子在句法上是相同的，但在语义上却存在差异，而语言模型需要具备卓越的语言能力和世界知识，才能准确评估序列的概率。

语言模型也可以做生成任务。如定义所示，语言模型p接受一个序列并返回一个概率来评估其好坏。我们也可以根据语言模型生成一个序列。最纯粹的方法是从语言模型$p$中以概率$p(x_{1:L})$进行采样，表示为：


$$
x_{1:L}∼p.
$$

如何在计算上高效地实现这一点取决于语言模型p的形式。实际上，我们通常不直接从语言模型中进行采样，这既因为真实语言模型的限制，也因为我们有时希望获得的不是一个“平均”的序列，而是更接近“最佳”序列的结果。


### 语言建模的基本原理
大语言模型的核心目标是学习一个语言的概率分布。给定一个词序列$$w_1,w_2,w_3...,w_N,$$大语言模型需要计算序列中每个词的条件概率，即给定前面的词序列，预测下一个词的概率。
在经典的语言建模中，模型的目标是通过条件概率来估计一个词序列的概率：
$$
P(w_1, w_2, \dots, w_N) = \prod_{i=1}^{N} P(w_i | w_1, w_2, \dots, w_{i-1})
$$这意味着每个词的出现是基于它前面的词出现的概率。模型的任务就是根据输入的上下文来计算每个词的条件概率。



### 1.1.1 自回归语言模型(Autoregressive language models)

将序列  $x_{1:L}$  的联合分布  $p(x_{1:L})$  的常见写法是使用概率的链式法则：

$$
p(x_{1:L}) = p(x_1) p(x_2 \mid x_1) p(x_3 \mid x_1, x_2) \cdots p(x_L \mid x_{1:L-1}) = \prod_{i=1}^L p(x_i \mid x_{1:i-1}).
$$

这里有一个基于文本的例子：

$$
\begin{align*} p({the}, {mouse}, {ate}, {the}, {cheese}) = \, & p({the}) \\ & p({mouse} \mid {the}) \\ & p({ate} \mid {the}, {mouse}) \\ & p({the} \mid {the}, {mouse}, {ate}) \\ & p({cheese} \mid {the}, {mouse}, {ate}, {the}). \end{align*}
$$

特别地，我们需要理解  $p(x_{i}∣x_{1:i−1})$  是一个给定前面的记号  $x_{1:i−1}$ 后，下一个记号  $x_{i}$  的条件概率分布。在数学上，任何联合概率分布都可以通过这种方式表示。然而，自回归语言模型的特点是它可以利用例如前馈神经网络等方法有效计算出每个条件概率分布  $p(x_{i}∣x_{1:i−1})$  。在自回归语言模型  $p$  中生成整个序列 $x_{1:L}$ ，我们需要一次生成一个词元(token)，该词元基于之前以生成的词元进行计算获得：

$$
\begin{aligned}
\text { for } i & =1, \ldots, L: \\
x_i & \sim p\left(x_i \mid x_{1: i-1}\right)^{1 / T},
\end{aligned}
$$

其中  $T≥0$  是一个控制我们希望从语言模型中得到多少随机性的温度参数：
- T=0：确定性地在每个位置 i 选择最可能的词元 $x_{i}$
- T=1：从纯语言模型“正常（normally）”采样
- T=∞：从整个词汇表上的均匀分布中采样
然而，如果我们仅将概率提高到  $1/T$  的次方，概率分布可能不会加和到 1。我们可以通过重新标准化分布来解决这个问题。我们将标准化版本  $p_{T}(x_{i}∣x_{1:i−1})∝p(x_{i}∣x_{1:i−1})^{1/T}$ 称为退火条件概率分布。例如：

$$
\begin{array}{cl}
p(\text { cheese })=0.4, & p(\text { mouse })=0.6 \\
p_{T=0.5}(\text { cheese })=0.31, & \left.p_{T=0.5} \text { (mouse }\right)=0.69 \\
\left.p_{T=0.2} \text { (cheese }\right)=0.12, & p_{T=0.2} \text { (mouse) }=0.88 \\
\left.p_{T=0} \text { (cheese }\right)=0, & \left.p_{T=0} \text { (mouse }\right)=1
\end{array}
$$

具体来说，这个温度参数会应用于每一步的条件概率分布 $p(x_{i}∣x_{1:i−1})$ ，将其幂变为  $1/T$ 。这意味着当  $T$  值较高时，我们会获得更平均的概率分布，生成的结果更具随机性；反之，当 $T$ 值较低时，模型会更倾向于生成概率较高的词元。

然而，有一个重要的注意事项：对于每一步的条件概率分布应用温度参数  $T$ ，并进行迭代采样，这种方法并不等同于（除非  $T=1$ ）从整个长度为 L 的序列的"退火"分布中一次性采样。换句话说，这两种方法在  $T≠1$  时会产生不同的结果。

"退火"这个术语来源于冶金学，其中热的金属会逐渐冷却以改变其物理性质。在这里，它类比的是对概率分布进行调整的过程。"退火"分布是通过将原始概率分布的每个元素都取幂  $1/T$ ，然后重新标准化得到的新分布。当 $T ≠ 1$ 时，这个过程会改变原始概率分布，因此从"退火"分布中采样得到的结果可能与对每一步的条件分布应用 T 并进行迭代采样的结果不同。

对于非自回归的条件生成，更一般地，我们可以通过指定某个前缀序列 $x_{1:i}$ （称为提示）并采样其余的  $x_{i+1:L}$ （称为补全）来进行条件生成。例如，生成 $T=0$ 的产生的：

$$
\underbrace{{the}, {mouse}, {ate}}_\text{prompt} \stackrel{T=0}{\leadsto} \underbrace{{the}, {cheese}}_\text{completion}.
$$

如果我们将温度改为 $T=1$ ，我们可以得到更多的多样性，例如，"its house" 和 "my homework"。我们将很快看到，条件生成解锁了语言模型通过简单地更改提示就能解决各种任务的能力。

### 1.1.2 总结

- 语言模型是序列  $x_{1:L}$ 的概率分布 p。
- 直观上，一个好的语言模型应具有语言能力和世界知识。
- 自回归语言模型允许有效地生成给定提示 $x_{1:i}$ 的补全 $x_{i+1:L}$。
- 温度可以用来控制生成中的变异量。